https://adventofcode.com/2020/day/11

--- Day 11: Seating System ---

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat layout might look like this:

L.LL.LL.LL  
LLLLLLL.LL  
L.L.L..L..  
LLLL.LL.LL  
L.LL.LL.LL  
L.LLLLL.LL  
..L.L.....  
LLLLLLLLLL  
L.LLLLLL.L  
L.LLLLL.LL  

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

    If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
    If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
    Otherwise, the seat's state does not change.

Floor (.) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##

After a second round, the seats with four or more occupied adjacent seats become empty again:

#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##

This process continues for three more rounds:

#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##

#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##

#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##

At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?


In [3]:
from collections import Counter

In [4]:
def seat_checker(seat_grid, seat_loc):
    '''All decisions are based on the 
        number of occupied seats adjacent to a 
        given seat (one of the eight positions
        immediately up, down, left, right, or diagonal from the seat)'''
    
    seat_row,seat_col = seat_loc[0],seat_loc[1]
    
#     print(f"Starting seat: ({seat_row},{seat_col})")
#     print(f"Starting seat status: {seat_grid[seat_row][seat_col]}")
    seats_nearby_status = Counter()
    
    left_seat = seat_grid[seat_row][seat_col-1]
    right_seat = seat_grid[seat_row][seat_col+1]
    up_seat = seat_grid[seat_row-1][seat_col]
    down_seat = seat_grid[seat_row+1][seat_col]
#     print(f"Left: {left_seat}")
#     print(f"Right: {right_seat}")
#     print(f"Up: {up_seat}")
#     print(f"Down: {down_seat}")
    #diags
    left_up_seat = seat_grid[seat_row-1][seat_col-1]
    left_down_seat = seat_grid[seat_row+1][seat_col-1]
    right_up_seat = seat_grid[seat_row-1][seat_col+1]
    right_down_seat = seat_grid[seat_row+1][seat_col+1]
    
#     print([left_seat,right_seat,up_seat,down_seat,left_up_seat,left_down_seat,right_up_seat,right_down_seat])
    
    for seat in [left_seat,right_seat,up_seat,down_seat,left_up_seat,left_down_seat,right_up_seat,right_down_seat]:
        seats_nearby_status[seat]+=1
        
    return int(seats_nearby_status["#"])

def create_buffer_grid(start_grid):
    ## Add buffer to seat grid
    #Buffer Left Column
    start_grid= np.insert(start_grid,0,".",axis=1)
    #Buffer Right Column
    start_grid= np.insert(start_grid,len(start_grid[0]),".",axis=1)
    #Buffer Top Row
    start_grid= np.insert(start_grid,0,["."*len(start_grid[0])],axis=0)
    # #Buffer Bottom Row
    start_grid= np.insert(start_grid,len(start_grid),["."*len(start_grid[0])],axis=0)
    return start_grid

def seat_status_checker(seat_status, num_seats_filled_nearby):
    #If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
    if seat_status == "L":
        if num_seats_filled_nearby == 0:
            return "#"
        else:
            return "L"
    #If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.        
    elif seat_status == "#":
        if num_seats_filled_nearby >= 4:
            return "L"
        else:
            return "#"
    
    else:
        return seat_status        


In [5]:
def run_grid_one_step(start_grid):
    next_layout = np.empty_like(start_grid)
    buffer_layout = create_buffer_grid(start_grid).copy()
    for row_idx, row in enumerate(start_grid):
        for seat_idx, seat_status in enumerate(row):
            new_status = seat_status_checker(seat_status\
                                     ,seat_checker(buffer_layout,(row_idx+1,seat_idx+1)))
            next_layout[row_idx,seat_idx]=new_status
    return next_layout

In [6]:
##### PART 1

In [7]:
with open('./data/day11_input.txt', 'r', newline='\n') as reader:
    seat_grid = reader.readlines()
seat_grid = np.array([list(x[:-1]) for x in seat_grid])
prev_state = np.empty_like(seat_grid)
runs = 0
x = (seat_grid==prev_state).all()

In [8]:
while not x:
    next_state = run_grid_one_step(seat_grid)
    prev_state = seat_grid.copy()
    seat_grid = next_state.copy()
    runs+=1
    x = (seat_grid==prev_state).all()
    if runs > 1000:
        print(".... Over 1000 runs")
        break
    
print("Static solution found...",(seat_grid=="#").sum(),"seats occupied")

Static solution found... 2254 seats occupied


-- Ok, so this is right. But I got stuck trying to adapt this to part 2. Going to put this down for a few hours or even the night and see if it's worthwhile adapting the above awfully dense code or just re-writing the entire step 2 from scratch (leaning towards the latter!).

I like the idea of instead of messing with a buffer row and column thing, just have a is_empty function (like the one here https://pastebin.com/LfgeneDM) that returns a 0 when it hits a KeyError. Much simpler to manage that way